In [1]:
!pip install pydub requests ffmpeg-python pandas

In [11]:
pip install pydub

  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1 -> 25.0.1
[notice] To update, run: C:\Users\rajas\pywavelets-env\Scripts\python.exe -m pip install --upgrade pip


In [13]:
import os
import time
import requests
import pandas as pd
from pydub import AudioSegment

radio_stations = {
    "Jazz Radio": "http://streaming.radio.co/s8bfa9f7e3/listen",
    "Rock Classics": "http://stream.whro.org/whro-rock",
    "Classical Music": "http://icecast.somafm.com/classical-mp3",
    "Hip-Hop Beats": "http://streaming.radio.co/s9a48e5a3c/listen",
    "Pop Hits": "http://us1.internet-radio.com:8105/stream",
    "Electronic Dance Music (EDM)": "http://listen.di.fm/public1/edm.pls",
    "Reggae Vibes": "http://uk7.internet-radio.com:8000/live.m3u",
    "Latin Rhythms": "http://stream.zeno.fm/0ypx8ghy3hhvv",
    "Blues Legends": "http://blues-radio-stream.live24.gr:80",
    "Indie Alternative": "http://streaming.live365.com/a35674",
}


# Output folder
output_folder = "music_audio_dataset"
os.makedirs(output_folder, exist_ok=True)

# CSV file to store metadata
csv_filename = os.path.join(output_folder, "metadata.csv")

# Function to record and save audio
def record_audio(station_name, url, duration=30):
    try:
        # Add stream headers to mimic browser request
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
        
        response = requests.get(url, stream=True, timeout=10, headers=headers)
        
        if response.status_code != 200:
            print(f"Failed to access {url}")
            return None

        # Save the raw audio data
        audio_path = os.path.join(output_folder, f"{station_name.replace(' ', '_')}_{int(time.time())}.mp3")
        with open(audio_path, "wb") as audio_file:
            start_time = time.time()
            for chunk in response.iter_content(chunk_size=1024):
                if time.time() - start_time > duration:
                    break
                audio_file.write(chunk)

        # Convert to WAV format
        audio_wav_path = audio_path.replace(".mp3", ".wav")
        audio = AudioSegment.from_mp3(audio_path)
        audio.export(audio_wav_path, format="wav")

        return station_name, audio_wav_path, time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time)), duration

    except Exception as e:
        print(f"Error recording {station_name}: {e}")
        return None

# Store metadata
metadata = []

for i in range(3):  # Capture multiple samples
    for station, url in radio_stations.items():
        result = record_audio(station, url, duration=30)
        if result:
            metadata.append(result)

# Save metadata to CSV
df = pd.DataFrame(metadata, columns=["Station Name", "File Path", "Timestamp", "Duration (sec)"])
df.to_csv(csv_filename, index=False)

print("Audio recording completed!")


Failed to access http://streaming.radio.co/s8bfa9f7e3/listen
Error recording Rock Classics: HTTPConnectionPool(host='stream.whro.org', port=80): Max retries exceeded with url: /whro-rock (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000001CC0CE313D0>: Failed to resolve 'stream.whro.org' ([Errno 11001] getaddrinfo failed)"))
Error recording Classical Music: HTTPConnectionPool(host='icecast.somafm.com', port=80): Max retries exceeded with url: /classical-mp3 (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000001CC0CE55AF0>: Failed to resolve 'icecast.somafm.com' ([Errno 11001] getaddrinfo failed)"))
Failed to access http://streaming.radio.co/s9a48e5a3c/listen
Error recording Pop Hits: HTTPConnectionPool(host='us1.internet-radio.com', port=8105): Max retries exceeded with url: /stream (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001CC0CE58FB0>: Failed to establish a new connection: [WinErro